<a href="https://colab.research.google.com/github/jumafernandez/BDM/blob/master/CyA_BD/Conceptos_y_Aplicaciones_Big_Data_UNLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **TRABAJO INTEGRADOR: CONCEPTOS Y APLICACIONES DE BIG DATA**

__Carrera:__ Espcialización en Inteligencia de Datos orientada a Big Data <br/>
__Docente:__ Waldo Hasperué  <br/>
__Estudiante:__ Juan Manuel Fernandez
<br/>  <br/>
__Resolución de Consultas utilizando PySpark:__ <br />

_Consulta 1:_ Puntaje histórico promedio de cada uno de los arqueros que jugaron en algún equipo de Uruguay. <br />

_Consulta 2:_ ¿Cuál es el nombre del equipo que salio campeón en la temporada 2015 de la primera división de Francia? __(optativo)__ <br />




### Preparación del entorno Pyspark y carga de datos

En primer lugar, cargamos los datos de fantasía desde un repositorio Github:



In [37]:
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/liga.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/equipo.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/jugador.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/encuentro.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/participacion_jugador.csv
!wget https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/goles.csv

--2019-09-30 22:13:45--  https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/liga.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 310 [text/plain]
Saving to: ‘liga.csv.1’

liga.csv.1          100%[===================>]     310  --.-KB/s    in 0s      

2019-09-30 22:13:45 (27.6 MB/s) - ‘liga.csv.1’ saved [310/310]

--2019-09-30 22:13:47--  https://raw.githubusercontent.com/jumafernandez/UNLP/master/CyA_BD/data/equipo.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358 [text/plain]
Saving to: ‘equipo.csv.1’

equip

Instalamos la librería pyspark, Java 8 y seteo las variables de entorno para que no devuelva error:

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.3.2-bin-hadoop2.7"


Ahora, debemos elegir la versión 8 de Java:

In [39]:
!update-alternatives --config java

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 2


También debo instalar alguna versión de Java:

Ahora, generamos el entorno PySpark cargando las librerías a utilizar, instanciamos el Spark Context y se da formato columnar a los RDD:

In [40]:
# Importo las librerias
from pyspark import SparkContext, SQLContext

# Genero el Spark Context y el SQLContext para la DB 
sc = SparkContext("local", "Simple App") 

# Cargo SQLContext, solo para instanciar dataframes, no para consultas
df = SQLContext(sc)

# Defino el path donde estan los archivos (Solo para máquina local) 
data_path = ''

# Cargo los RDD con los datos de los archivos y acomodo los datos
RDD_liga = sc.textFile(data_path + "liga.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), int(d[1]), d[2].replace('"', ''), d[3].replace('"', '')))
RDD_liga.first()

RDD_equipo = sc.textFile(data_path + "equipo.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), d[1].replace('"', ''), d[2].replace('"', ''), d[3].replace('"', ''), d[4].replace('"', '')))
RDD_equipo.first()

RDD_jugador = sc.textFile(data_path + "jugador.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), d[1].replace('"', ''), int(d[2]), d[3].replace('"', '')))
RDD_jugador.first()

RDD_encuentro = sc.textFile(data_path + "encuentro.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), int(d[1]), d[2].replace('"', ''), int(d[3]), int(d[4]), d[5].replace('"', ''), float(d[6]), int(d[7])))
RDD_encuentro.first()

RDD_participacion_jugador = sc.textFile(data_path + "participacion_jugador.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id_jugador"').\
           map(lambda d: (int(d[0]), int(d[1]), int(d[2]), int(d[3]), int(d[4]), int(d[5]), int(d[6])))
RDD_participacion_jugador.first()

RDD_goles = sc.textFile(data_path + "goles.csv").\
           map(lambda line: line.split(";")).\
           filter(lambda d: d[0] != '"id"').\
           map(lambda d: (int(d[0]), int(d[1]), int(d[2]),d[3].replace('"', ''), int(d[4])))


RDD_goles.first()


(1, 2, 2, 'CABEZA', 90)

Luego, los transformo a dataframes por dos cuestiones centrales:
1. _Conflictos con el Join en los RDD:_ Cuando intento aplicar un join sobre un RDD puro, me proyecta solo un subconjuto de los atributos.
2. _Comodidad:_ trabajando con dataframes, puedo referenciar los datos por nombre de columnas, lo cual brinda mayor expresividad y facilidad para la resolución de las consultas.

Independientemente de esto, no se utilizarán las otras características de SparkSQL (consultas SQL) en la resolución del trabajo.

In [68]:
# Creo los dataframes
df_ligas = df.createDataFrame(RDD_liga,['id','temporada', 'division', 'pais'])
df_equipos =  df.createDataFrame(RDD_equipo,['id','nombre', 'pais', 'ciudad', 'fundacion'])
df_jugadores = df.createDataFrame(RDD_jugador,['id','nombre', 'edad', 'posicion'])
df_encuentros = df.createDataFrame(RDD_encuentro,['id','id_liga', 'fecha', 'id_equipo_local', 'id_equipo_visitante', 'estadio', 'recaudacion', 'localidades_vendidas'])
df_participaciones = df.createDataFrame(RDD_participacion_jugador,['id_jugador','id_equipo', 'id_partido', 'puntaje_recibido', 'cantidad_amarillas', 'cantidad_rojas', 'minutos_jugados'])
df_goles = df.createDataFrame(RDD_goles,['id', 'id_jugador', 'id_partido', 'tipo_gol', 'minuto_gol'])

df_encuentros.show()


+---+-------+----------+---------------+-------------------+------------------+-----------+--------------------+
| id|id_liga|     fecha|id_equipo_local|id_equipo_visitante|           estadio|recaudacion|localidades_vendidas|
+---+-------+----------+---------------+-------------------+------------------+-----------+--------------------+
|  2|      1|2019-05-07|              1|                  2|ALBERTO J. ARMANDO|  2500000.0|               60000|
|  3|      1|2019-10-01|              2|                  1|  VESPUCIO LIBERTI|  3000000.0|               40000|
+---+-------+----------+---------------+-------------------+------------------+-----------+--------------------+



### Resolución Consulta 1: Puntaje histórico promedio de cada uno de los arqueros que jugaron en algún equipo de Uruguay.

Para comenzar con la resolución, definimos parámetros para flexibilizar la consulta (posición y país):

In [0]:
posicion = 'ARQUERO'
pais_filtro = 'ARGENTINA'

Tomo los equipos de la variable _pais_filtro_ y los jugadores que juegan en el contenido de la variable _posicion_:


In [0]:
equipos_pais = df_equipos.filter(df_equipos.pais==pais_filtro)
jugadores_posicion = df_jugadores.filter(df_jugadores.posicion==posicion)

A continuación, resuelvo el resto de la consulta tomando las participaciones de los arqueros en equipos uruguayos y agrupando las instancias aplicando la función de agregación MEAN:

In [44]:
# Joineo ambas cosas
participaciones = jugadores_posicion.join(df_participaciones, jugadores_posicion.id==df_participaciones.id_jugador)

# Tomo las columnas que me interesan y muestro el resultado agrupando
jugadores_puntaje = participaciones.select('nombre', 'puntaje_recibido')
jugadores_puntaje.groupby('nombre').agg({'puntaje_recibido': 'mean'}).show()

+----------------+---------------------+
|          nombre|avg(puntaje_recibido)|
+----------------+---------------------+
|  ARMANI, FRANCO|                  4.0|
|ANDRADA, ESTEBAN|                  8.0|
+----------------+---------------------+



### Resolución Consulta ¿Cuál es el nombre del equipo que salio campeón en la temporada 2015 de la primera división de Francia?

Para comenzar con la resolución, definimos parámetros para flexibilizar la consulta (temporada, división y país):

In [0]:
# Parametros de búsqueda
temporada = 2019
division = 'PRIMERA A'
pais = 'ARGENTINA'

En primer instancia, recupero el _id_ de la liga que corresponde a la temporada, division y pais de la consulta:


In [46]:
RDD_liga_q2 = RDD_liga.filter(lambda d: d[3] == pais).\
                      filter(lambda d: d[1] == temporada).\
                      filter(lambda d: d[2] == division)

id_liga_buscada = RDD_liga_q2.first()[0]

print('La liga buscada tiene id ' + str(id_liga_buscada))

La liga buscada tiene id 1


A continuación, selecciono todas las participaciones de jugadores en encuentros con el id de la liga recuperada:

In [47]:
encuentros_liga = df_encuentros.filter(df_encuentros.id_liga==id_liga_buscada)
participaciones_liga = df_participaciones.join(encuentros_liga, df_participaciones.id_partido==encuentros_liga.id, how='left')
participaciones_liga = participaciones_liga.filter(participaciones_liga.id_liga==id_liga_buscada)

# Muestro los datos recuperados
participaciones_liga.show()

+----------+---------+----------+----------------+------------------+--------------+---------------+---+-------+----------+---------------+-------------------+----------------+-----------+--------------------+
|id_jugador|id_equipo|id_partido|puntaje_recibido|cantidad_amarillas|cantidad_rojas|minutos_jugados| id|id_liga|     fecha|id_equipo_local|id_equipo_visitante|         estadio|recaudacion|localidades_vendidas|
+----------+---------+----------+----------------+------------------+--------------+---------------+---+-------+----------+---------------+-------------------+----------------+-----------+--------------------+
|         1|        1|         3|               8|                 0|             0|             90|  3|      1|2019-10-01|              2|                  1|VESPUCIO LIBERTI|  3000000.0|               40000|
|         2|        1|         3|               6|                 2|             1|             90|  3|      1|2019-10-01|              2|                  1|V

Ahora _joineo_ las participaciones de los jugadores con los goles que es lo que me interesa:

In [74]:
# Primero, cambio el nombre de id_partido por un problema Python con columnas repetidas
goles = df_goles.selectExpr('id_jugador', 'id_partido as id_partido_gol')

# Realizo el join
goles_por_participaciones=goles.join(participaciones_liga, (goles.id_jugador==participaciones_liga.id_jugador) & (goles.id_partido_gol==participaciones_liga.id_partido))

# Me quedo solo con los atributos que necesito
goles_por_equipo_partido = goles_por_participaciones.selectExpr('id_equipo', 'id_partido')

goles_por_equipo_partido.show()

+---------+----------+
|id_equipo|id_partido|
+---------+----------+
|        1|         3|
|        1|         2|
|        1|         2|
+---------+----------+



Acto seguido, agrupo los goles por equipo y partido:

In [76]:
goles_por_equipo_partido = goles_por_equipo_partido.groupBy('id_equipo', 'id_partido').count()

goles_por_equipo_partido.show()

+---------+----------+-----+
|id_equipo|id_partido|count|
+---------+----------+-----+
|        1|         3|    1|
|        1|         2|    1|
+---------+----------+-----+



Ahora que ya tengo los goles por participación, voy a joinear los datos con los partidos que se jugaron en la liga consultada:

In [0]:
# Me quedo con las columnas que necesito
encuentros = encuentros_liga.selectExpr('id', 'id_equipo_local', 'id_equipo_visitante') # Me quedo solo con los atributos que necesito

# Joineo con id_equipo_local
encuentros = encuentros.join(goles_por_equipo_partido, (encuentros.id==goles_por_equipo_partido.id_partido) & (encuentros.id_equipo_local==goles_por_equipo_partido.id_equipo))
encuentros = encuentros.withColumnRenamed('count', 'goles_equipo_local')
encuentros = encuentros.selectExpr('id', 'id_equipo_local', 'id_equipo_visitante', 'goles_equipo_local') # Me quedo solo con los atributos que necesito

# Joineo con id_equipo_visitante
encuentros = encuentros.join(goles_por_equipo_partido, (encuentros.id==goles_por_equipo_partido.id_partido) & (encuentros.id_equipo_visitante==goles_por_equipo_partido.id_equipo))
encuentros = encuentros.withColumnRenamed('count', 'goles_equipo_visitante')

# Me quedo unicamente con los id de partido, equipos y con los goles por equipo
encuentros = encuentros.selectExpr('id', 'id_equipo_local', 'id_equipo_visitante', 'goles_equipo_local', 'goles_equipo_visitante') # Me quedo solo con los atributos que necesito


Me traigo algunas funciones pyspark para manipular los _dataframe_:

In [0]:
from pyspark.sql.functions import when, lit, desc
from pyspark.sql import functions as F

A continuación, ya que tengo los resultados de cada partido, le asigno puntos a cada resultado. Para ello, tengo que generar dos atributos nuevos, uno denominado __puntos_equipo_local__ y otro __puntos_equipo_visitante__:

In [0]:
# Antes, para que funcione, reemplazo los None de los equipos que no hicieron goles en los partidos por 0
df_resultados = encuentros.withColumn('goles_equipo_local', when(encuentros.goles_equipo_local.isNull(), 
lit(0)).otherwise(encuentros.goles_equipo_local)).withColumn('goles_equipo_visitante', when(encuentros.goles_equipo_visitante.isNull(), 
lit(0)).otherwise(encuentros.goles_equipo_visitante))

# Ahora veo cuantos puntos se quedaron por partido equipos locales y visitantes
df_resultados = df_resultados.withColumn('puntos_equipo_local',
                                        F.when((F.col("goles_equipo_local") > F.col("goles_equipo_visitante")), 3)\
                                        .when((F.col("goles_equipo_local") == F.col("goles_equipo_visitante")), 1)\
                                        .otherwise(0))

df_resultados = df_resultados.withColumn('puntos_equipo_visitante',
                                        F.when((F.col("goles_equipo_visitante") > F.col("goles_equipo_local")), 3)\
                                        .when((F.col("goles_equipo_visitante") == F.col("goles_equipo_local")), 1)\
                                        .otherwise(0))

Realizo una operación de union de id_equipo_local y id_equipo_visitante y aplano la lista, todo para luego armar la tabla de posiciones de la liga:

In [0]:
# Elijo solo los atributos que necesito
df_puntos_local = df_resultados.selectExpr('id', 'id_equipo_local as id_equipo', 'puntos_equipo_local as puntos')
df_puntos_visitante = df_resultados.selectExpr('id', 'id_equipo_visitante as id_equipo', 'puntos_equipo_visitante as puntos')

# Union
df_puntaje_partido = df_puntos_local.union(df_puntos_visitante)

Genero la tabla de posiciones, para ello agrupo puntos por equipo y luego joineo la tabla con el dataframe _equipos_ a efectos de traerme los nombres de los mismos:

In [0]:
# Agrupo por id_equipo, sumo los puntos obtenidos y ordeno (esta es la tabla de posiciones)
tabla_posiciones = df_puntaje_partido.groupby('id_equipo').agg({'puntos': 'sum'})\
                   .sort(desc('sum(puntos)')).withColumnRenamed('sum(puntos)', 'Puntos')

# Hago un join con el df equipos para traer el nombre de los equipos
tabla_posiciones=tabla_posiciones.join(df_equipos, tabla_posiciones.id_equipo==df_equipos.id)
tabla_posiciones=tabla_posiciones.selectExpr('nombre as Equipo', 'Puntos')

Muestro la Tabla de Posiciones:

In [92]:
tabla_posiciones.show()

Py4JJavaError: ignored

In [0]:
Muestro al Campeón:

In [65]:
# Con first me quedo con el campeon
tabla_posiciones.first()

Py4JJavaError: ignored

Detengo el Spark Context:

In [0]:
sc.stop()



-- <br/>
Juan Manuel Fernandez <br />
___Conceptos y Aplicaciones de Big Data - UNLP___